In [2]:
import numpy as np
from abc import abstractmethod
from sklearn.base import BaseEstimator
from sklearn.neighbors import NearestCentroid
from sklearn.covariance import ShrunkCovariance
from sklearn import svm


class Classifier:

    @abstractmethod
    def fit(self, X, y):
        pass

    @abstractmethod
    def predict(self, X):
        pass
    

class ClassifEuclid(Classifier):
    def __init__(self):
        """Constructor de la clase"""
        self.centroids = None
        
    def fit(self,X, y):
        """Entrena el clasificador
        X: matriz numpy, cada fila es un dato, cada columna una medida del vector de caracteristicas.
        y: vector de etiquetas de clase, tantos elementos como filas en X.
        Retorna objeto clasificador"""

        assert X.ndim == 2 and X.shape[0] == len(y)
        
        # Find the centroids
        self.centroids = np.array([np.mean(X[y == t], axis = 0) for t in np.unique(y)])

        #print(f"centroids: {self.centroids}")
        return self

    def decision_function(self, X):
        """Estima el grado de pertenencia de todos los datos a todas las clases 
        X: matriz numpy, cada fila es un dato, cada columna una medida del vector de caracteristicas. 
        Retorna una matriz, con tantas filas como datos y tantas columnas como clases tenga
        el problema, cada fila almacena los valores pertenencia de un dato a cada clase""" 
        assert self.centroids is not None, "Error: The classifier needs to be fitted. Please call fit(X, y) method."
        assert X.ndim == 2 and X.shape[1] == self.centroids.shape[1]        
        
        # Distance: 150,1,4 - 1,3,4 -> 150,3,4 -> 150,3
        res = np.linalg.norm(X[:, np.newaxis, :] - self.centroids[np.newaxis, :, :], axis = 2)
        
        #print(f"distances: {res}")
        return res
    
    def predict(self, X):
        """Predice una clase para cada dato. La clase retornada debe ser un entero.
        X: matriz numpy donde cada fila es un dato, cada columna una medida.
        Retorna un vector con la clase predicha para cada dato"""
        
        # Find the distances to the centroids
        distances = self.decision_function(X)

        # Find the shortest distance
        res = np.argmin(distances, axis = 1)
        
        #print(f"Predicted: {res}")
        return res

    
def test(X, Y):
    return np.sum(X != Y) # Number of different values between X and Y


color_red = np.array([255, 0, 0])
color_green = np.array([0, 255, 0])
color_blue = np.array([0, 0, 255])
paleta = np.array([[0, 0, 255], [255, 255, 255], [255, 0, 0]])

im_border = np.zeros((150, 320), dtype = np.bool)

for i in range(150):
    for j in range(320):
        if i<3 or i>147 or j<3 or j>317:
            im_border[i,j] = True
        else:
            im_border[i,j] = False

In [3]:
from imageio import imread, imsave

# Cargamos los datos para entrenar
marked = imread("imgs/train/marked.png")
norm = imread("imgs/train/norm.png")

red_mask = np.sum(marked==color_red, axis=2) == 3
green_mask = np.sum(marked==color_green, axis=2) == 3
blue_mask = np.sum(marked==color_blue, axis=2) == 3


# Datos de refuerzo para la marca cuando le da la luz
marked_2 = imread("imgs/train/marked_2.png")
norm_2 = imread("imgs/train/norm_2.png")

red_mask_2 = np.sum(marked_2==color_red, axis=2) == 3

X_marca = norm[red_mask]
X_marca_2 = norm_2[red_mask_2]
X_fondo = norm[green_mask]
X_linea = norm[blue_mask]

n_marca,_ = X_marca.shape
n_marca_2,_ = X_marca_2.shape
n_fondo,_ = X_fondo.shape
n_linea,_ = X_linea.shape

y_marca = np.zeros((n_marca),dtype=int)
y_marca_2 = np.zeros((n_marca_2),dtype=int)
y_fondo = np.ones((n_fondo),dtype=int)
y_linea = np.ones((n_linea),dtype=int) + 1

# Entrenamiento normal
X = np.concatenate((X_marca_2, X_fondo, X_linea), axis=0)
y = np.concatenate((y_marca_2, y_fondo, y_linea), axis=0)

# Pruebas
#X = np.concatenate((X_marca, X_marca_2, X_fondo, X_linea), axis=0)
#y = np.concatenate((y_marca, y_marca_2, y_fondo, y_linea), axis=0)

# Elegir el clasificador
classifier = ClassifEuclid()

# Entrenar el clasificador
classifier.fit(X,y)

In [4]:
import cv2
from matplotlib import pyplot as plt
from operator import itemgetter
import math

def rg_norm_img(img):
    return img / np.sum(img, axis=2)[:, :, np.newaxis] * 255


def segment(img):
    
    img_h,img_w,_ = img.shape
    
    img_norm = rg_norm_img(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    
    img_aux = np.reshape(img_norm, (-1, 3))   
    predicted = classifier.predict(img_aux)
    img_res_paleta = paleta[np.reshape(predicted,(img_h,img_w))]
    img_res = np.reshape(predicted,(img_h,img_w))
    
    return img_res,img_res_paleta


def define_case(im, img_seg):

    # Separo los bordes a partir de las etiquetas de segmentación y extraigo los contornos
    img_grey_line = (img_seg==2).astype("uint8")*255
    border = np.logical_and(img_grey_line, im_border).astype("uint8")*255
    contList,hier = cv2.findContours(border,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    
    # Pinto los contornos
    #img_cont = cv2.drawContours(np.float32(im), contList, -1, (0,255,0))
    img_cont = cv2.drawContours(np.float32(im), [], -1, (0,255,0))
    
    # Calculolos centros de los contornos
    cent = []
    for c in contList:
        M = cv2.moments(c)
        try:
            center = [M["m10"]/M["m00"], M["m01"] / M["m00"]]
        except:
            center = [M["m10"]/1, M["m01"]/1]
        cent.append(center)
        
    # Pintamos las salidas y la entrada
    for c in cent:
        img_cont = cv2.circle(img_cont, (int(c[0]),int(c[1])), 8, (0,32,255), 15)
    
    # Defino el caso en funcion del numero de contornos
    n = len(contList)
    case = -1
    
    if n < 3:
        d = [cent[0][0]-cent[1][0], cent[0][1]-cent[1][1]]
        
        if d[0] < -50:
            text = cv2.putText(img_cont, 'Curva a la derecha', org, font, fontScale, color, thickness, cv2.LINE_AA)
            case = 1 
        elif d[0] > 50:
            text = cv2.putText(img_cont, 'Curva a la izquerda', org, font, fontScale, color, thickness, cv2.LINE_AA)
            case = 0 
        else:
            text = cv2.putText(img_cont, 'Linea recta', org, font, fontScale, color, thickness, cv2.LINE_AA)
            case = 0

    elif n < 4:
        text = cv2.putText(img_cont, 'Bifurcacion en Y', org, font, fontScale, color, thickness, cv2.LINE_AA)
        case = 3
        
    elif n < 5:
        text = cv2.putText(img_cont, 'Cruce en X', org, font, fontScale, color, thickness, cv2.LINE_AA)
        case = 4
        
    return img_cont, case, cent


def study_arrow(im, img_seg, case, arrow, acc):
    
    d = "None"
    
    # Construyo imágenes a partir de las etiquetas de segmentación y extraigo los contornos
    img_grey_mark = (img_seg==0).astype("uint8")*255 
    contList,hier = cv2.findContours(img_grey_mark,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    
    # Cojo el contorno mas grande con un área mayor a 200
    if len(contList) != 0:
        contList = [max(contList, key = cv2.contourArea)]
        area = cv2.contourArea(contList[0])
        
        if area < 200:
            contList = []
    
    # Pinto los contornos
    #img_cont = cv2.drawContours(np.float32(im), contList, -1, (0,255,0))
    
    # Evaluamos la flecha si la hay
    if len(contList) > 0:
        # Calculo el centro de gravedad
        M = cv2.moments(contList[0])
        try:
            gravity_center = [M["m10"]/M["m00"], M["m01"] / M["m00"]]
        except:
            gravity_center = [M["m10"]/1, M["m01"]/1]
        
        #img_cont = cv2.circle(img_cont, (int(gravity_center[0]),int(gravity_center[1])), 2, (0,32,255), 2)
                
        # Calculo el centro del rectangulo que contiene la flecha
        rect = cv2.minAreaRect(contList[0])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        #im = cv2.drawContours(im,[box],0,(0,32,255),2)
        
        M = cv2.moments(box)
        try:
            center = [M["m10"]/M["m00"], M["m01"] / M["m00"]]
        except:
            center = [M["m10"]/1, M["m01"]/1]
        
        #img_cont = cv2.circle(img_cont, (int(center[0]),int(center[1])), 2, (0,255,255), 2)
        
        # Calculo la direccion en funcion de los centros calculados
        v_dir = [center[0]-gravity_center[0], center[1]-gravity_center[1]]
        
        if abs(v_dir[0]) > abs(v_dir[1]):
            if v_dir[0] > 0:
                acc[0] = acc[0] + 1
            else:
                acc[1] = acc[1] + 1
        else:
            if v_dir[1] >= 0:
                acc[2] = acc[2] + 1
            else:
                acc[3] = acc[3] + 1
        
        # Establezco la direccion de la flecha
        if acc[0] > acc[1] and acc[0] > acc[2] and acc[0] >= acc[3]:
            d = "izquierda"
            arrow = 0
        elif acc[1] >= acc[0] and acc[1] > acc[2] and acc[1] >= acc[3]:
            d = "derecha"
            arrow = 1
        elif acc[2] >= acc[0] and acc[2] >= acc[1] and acc[2] >= acc[3]:
            d = "frente"
            arrow = 2
        elif acc[3] > acc[0] and acc[3] > acc[1] and acc[3] > acc[2]:
            d = "atras"
            arrow = 3

    return img_cont, d, arrow, acc        
    

def study_marks(img_cont, img_seg, im_seg_paleta, acc):
    
    # Construyo imágenes a partir de las etiquetas de segmentación y extraigo los contornos
    img_grey_mark = (img_seg==0).astype("uint8")*255 
    contList,hier = cv2.findContours(img_grey_mark,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    
    # Cojo el contorno mas grande
    if len(contList) != 0:
        contList = [max(contList, key = cv2.contourArea)]
        area = cv2.contourArea(contList[0])
        
        if area < 50:
            contList = []
            
    #img_cont = cv2.drawContours(np.float32(img_cont), contList, -1, (0,255,0))
    
    if len(contList) != 0:
        des =  orb_descriptors(contList[0], img_grey_mark)

        if des is not None:
            mark = neigh.predict([des])
            acc[int(mark)] = acc[int(mark)] + 1
            
            # Establezco la direccion de la flecha
            if acc[0] > acc[1] and acc[0] > acc[2] and acc[0] >= acc[3]:
                mark_text = "Hombre"
            elif acc[1] >= acc[0] and acc[1] > acc[2] and acc[1] >= acc[3]:
                mark_text = "Escalera"
            elif acc[2] >= acc[0] and acc[2] >= acc[1] and acc[2] >= acc[3]:
                mark_text = "Telefono"
            elif acc[3] > acc[0] and acc[3] > acc[1] and acc[3] > acc[2]:
                mark_text = "Mujer"
            
            text = cv2.putText(img_cont, f'marca: {mark_text}', org_5, font, fontScale, color, thickness, cv2.LINE_AA)
            
        else:
            acc = [0,0,0,0]
            
    return img_cont, acc


def orb_descriptors(cont, im):
    
    orb = cv2.ORB_create()
    ellip = cv2.fitEllipse(cont)
    cen, ejes, angulo = np.array(ellip[0]), np.array(ellip[1]), ellip[2]
    if angulo > 90:
        angulo -= 180
        
    kp = cv2.KeyPoint(cen[0], cen[1], np.mean(ejes)*1.3, angulo - 90)
    lkp, des = orb.compute(im, [kp])
    
    if des is not None:
        return np.unpackbits(des).T
    return None


def eval_exit(img_cont, cent, arrow):

    if arrow == 0:
        exit = min(cent, key=itemgetter(0))
    elif arrow == 1:
        exit = max(cent, key=itemgetter(0))
    elif arrow == 2:
        exit = min(cent, key=itemgetter(1))
    elif arrow == 3:
        exit = max(cent, key=itemgetter(1))
    
    img_cont = cv2.circle(img_cont, (int(exit[0]),int(exit[1])), 8, (255,0,255), 15)
    
    if exit[1] < 60:
        eX = 60
    else:
        eX = exit[1]

    vel = round((151 - exit[1]) / 150, 2)
    ang = round(((exit[0] - 160) * eX) / (320 * 75), 2)
    
    cv2.putText(img_cont, f"velocidad: {vel}", org_3, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.putText(img_cont, f"angulo: {ang}", org_4, font, fontScale, color, thickness, cv2.LINE_AA)
        
    return img_cont


def vid_classifier(vid):
    
    capture = cv2.VideoCapture(vid)
    ret,im = capture.read()

    img_h,img_w,_=im.shape
    out = cv2.VideoWriter('videos/output/clasificado_2.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (img_w,img_h))
    
    while(ret):
        # Nuevo video
        img_seg = segment(im)
        out.write(img_seg.astype('uint8'))
        
        ret,im = capture.read()
 
    capture.release()
    out.release()

In [5]:
# Cargamos los datos para probar
marked = imread("imgs/train/test_marked.png")
norm = imread("imgs/train/test_norm.png")

red_mask = np.sum(marked==color_red, axis=2) == 3
green_mask = np.sum(marked==color_green, axis=2) == 3
blue_mask = np.sum(marked==color_blue, axis=2) == 3

X_marca = norm[red_mask]
X_fondo = norm[green_mask]
X_linea = norm[blue_mask]

n_marca,_ = X_marca.shape
n_marca_2,_ = X_marca_2.shape
n_fondo,_ = X_fondo.shape
n_linea,_ = X_linea.shape

y_marca = np.zeros((n_marca),dtype=int)
y_fondo = np.ones((n_fondo),dtype=int)
y_linea = np.ones((n_linea),dtype=int) + 1

# Entrenamiento normal
X = np.concatenate((X_marca_2, X_fondo, X_linea), axis=0)
y_res = np.concatenate((y_marca_2, y_fondo, y_linea), axis=0)

y = classifier.predict(X)

errors = test(y, y_res)
total = len(y)

print(f"El clasificador ha cometido un total de {errors} errores al analizar {total} elementos, por lo tanto tiene un {round((((total-errors)/total)*100),4)}% de acierto")

#_,img_res_paleta = segment(norm)
#plt.imshow(img_res_paleta)
#plt.show()
#imsave(f'imgs/train/test_res.png',img_res_paleta.astype('uint8'))


El clasificador ha cometido un total de 2 errores al analizar 57756 elementos, por lo tanto tiene un 99.9965% de acierto


In [10]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import joblib

X = np.empty((28,256))
y = np.array([0, 0, 0, 0, 0, 0, 0 ,1 ,1 ,1 ,1 ,1 ,1 ,1, 2 ,2 ,2 ,2 ,2 ,2 ,2 ,3 ,3 ,3 ,3 ,3 ,3 ,3])

for i in range(1,29):
    
    im = imread(f"imgs/marcas/{i}.png")
        
    img_seg, img_seg_paleta = segment(im)
    
    img_grey_mark = (img_seg==2).astype("uint8")*255
    
    contList,hier = cv2.findContours(img_grey_mark,cv2.RETR_LIST,cv2.CHAIN_APPROX_NONE)
    
    # Cojo el contorno mas grande
    if len(contList) != 0:
        contList = [max(contList, key = cv2.contourArea)]
        area = cv2.contourArea(contList[0])
        
        if area < 50:
            contList = []
    
    img_cont = cv2.drawContours(np.float32(img_grey_mark), contList, -1, (255,0,255))
    
    X[i-1] = orb_descriptors(contList[0], img_grey_mark)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)      
    
neigh = KNeighborsClassifier(1, metric="euclidean")
neigh.fit(X_train, y_train)
joblib.dump(neigh, './clf/clf.pkl')

y_res = neigh.predict(X_test)
errors = test(y_test, y_res)
total = len(y_test)

print(f"El clasificador ha cometido un total de {errors} errores al analizar {total} elementos, por lo tanto tiene un {round((((total-errors)/total)*100),4)}% de acierto")

<ipython-input-4-084e151e2883>:7: RuntimeWarning: invalid value encountered in true_divide
  return img / np.sum(img, axis=2)[:, :, np.newaxis] * 255


El clasificador ha cometido un total de 1 errores al analizar 6 elementos, por lo tanto tiene un 83.3333% de acierto


In [9]:
import cv2
from imageio import imread, imsave
    
# Text
font = cv2.FONT_HERSHEY_SIMPLEX
org = (15, 15)
org_2 = (15,30)
org_3 = (15,45)
org_4 = (15,60)
org_5 = (15,75)
fontScale = 0.5
color = (0, 0, 0)
thickness = 1

#Captura de imagenes 
capture = cv2.VideoCapture('videos/video-2.avi')
out = cv2.VideoWriter('videos/output/clasificado_2.avi', cv2.VideoWriter_fourcc(*'XVID'), 20.0, (320, 150))

# Parametros auxiliares
key = ""
arrow = 2
acc = [0,0,0,0]
acc_mark = [0,0,0,0]

while (key != ord('q')):
    
    # Leo una imagen del video
    ret,im = capture.read()
    
    # Si no hay más imagenes paramos
    if not ret:
        print('Se acabo el video.')
        cv2.destroyAllWindows()
        break
        
    # Recorto la imagen
    im = im[90:,:]
    
    # Segmento la imagen
    img_seg, img_seg_paleta = segment(im)

    # Estudiamos en que caso nos encontramos
    img_cont, case, cent = define_case(img_seg_paleta, img_seg)
    
    # Estudiamos las marcas del fotograma
    d = "None"
    
    if case < 3:
        arrow = 2
        acc = [0,0,0,0]
        img_cont, acc_mark = study_marks(img_cont, img_seg, img_seg_paleta, acc_mark)
    else:
        img_cont, d, arrow, acc = study_arrow(img_cont, img_seg, case, arrow, acc)
    
    text = "Flecha: " + d 
    cv2.putText(img_cont, text, org_2, font, fontScale, color, thickness, cv2.LINE_AA)
    
    img_cont = eval_exit(img_cont, cent, arrow)
    
    # Muestro la imagen en una ventala OpenCV
    cv2.imshow("Captura", img_cont)
    out.write(img_cont.astype('uint8'))

    # Pausa si se pulsa P, stop si se pulsa Q
    key = cv2.waitKey(35)
    if key == ord('p'):
        key = cv2.waitKey(-1) # Espero hasta que se pulse otra tecla

# Cierro la ventana 
cv2.destroyAllWindows()
capture.release()
out.release()